# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [13]:
#Your code here
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm

In [3]:
# Load the dataset
df = pd.read_csv('homepage_actions.csv')

In [6]:
#Your code here
# Preview the data
print(df.head())

# Check unique actions and groups
print(df['action'].value_counts())
print(df['group'].value_counts())

# How many unique viewers and clickers?
viewers = set(df[df['action'] == 'view']['id'])
clickers = set(df[df['action'] == 'click']['id'])
print(f"Number of viewers: {len(viewers)}")
print(f"Number of clickers: {len(clickers)}")

# How many viewers also clicked?
viewed_and_clicked = viewers & clickers
print(f"Number of viewers who also clicked: {len(viewed_and_clicked)}")

# Check for overlap between control and experiment groups
user_groups = df.groupby('id')['group'].nunique()
overlap = user_groups[user_groups > 1]
print(f"Number of users in both groups: {len(overlap)}")


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
view     6328
click    1860
Name: action, dtype: int64
control       4264
experiment    3924
Name: group, dtype: int64
Number of viewers: 6328
Number of clickers: 1860
Number of viewers who also clicked: 1860
Number of users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
#Your code here
# Remove users in both groups (if any)
user_groups = df.groupby('id')['group'].nunique()
overlap_ids = user_groups[user_groups > 1].index
df = df[~df['id'].isin(overlap_ids)]

# Pivot to get views and clicks per group
views = df[df['action'] == 'view'].groupby('group')['id'].nunique()
clicks = df[df['action'] == 'click'].groupby('group')['id'].nunique()

# Prepare data for z-test
successes = [clicks['control'], clicks['experiment']]
nobs = [views['control'], views['experiment']]

# Run two-proportion z-test
z_stat, p_val = proportions_ztest(successes, nobs)
print(f"z-statistic: {z_stat:.4f}")
print(f"p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Statistically significant difference between groups.")
else:
    print("No statistically significant difference between groups.")

z-statistic: -2.6186
p-value: 0.0088
Statistically significant difference between groups.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
#Your code here

# Control group click-through rate
ctr_control = clicks['control'] / views['control']

# Expected number of clicks in experiment group if it had control's CTR
expected_clicks_exp = ctr_control * views['experiment']

print(f"Control group CTR: {ctr_control:.4f}")
print(f"Expected number of clicks in experiment group (if same CTR as control): {expected_clicks_exp:.2f}")

Control group CTR: 0.2797
Expected number of clicks in experiment group (if same CTR as control): 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [12]:
#Your code here

# Actual number of clicks in experiment group
actual_clicks_exp = clicks['experiment']

# Variance and standard deviation for binomial
variance = views['experiment'] * ctr_control * (1 - ctr_control)
std_dev = np.sqrt(variance)

# z-score calculation
z = (actual_clicks_exp - expected_clicks_exp) / std_dev

print(f"Actual clicks in experiment group: {actual_clicks_exp}")
print(f"Standard deviation: {std_dev:.2f}")
print(f"z-score: {z:.4f}")

Actual clicks in experiment group: 928
Standard deviation: 24.57
z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [14]:
#Your code here
# Calculate two-tailed p-value from z-score
p_value = 2 * (1 - norm.cdf(abs(z)))
print(f"p-value: {p_value:.4f}")

p-value: 0.0002


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, this result matches the previous statistical test. Both approaches yield a very small p-value, indicating a statistically significant difference between the control and experiment groups. This confirms that the observed difference in click-through rates is unlikely to be due to random chance.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.